# Langchain quickstart

In this notebook you will see how easy it is to create your first chain usiung EPAM Dial as a LLM proxy. In just few steps we will use EPAM Dial to send messages to LLM. In next step we will introduce output parser to show how to use chains.
Notebook based on https://python.langchain.com/v0.1/docs/get_started/quickstart/ adapted to EPAM Dial.

To create LLM instance (in this case EPAM Dial) use:

In [1]:
import os
from langchain_openai import AzureChatOpenAI

epam_dial = AzureChatOpenAI(
        api_key         = os.environ['OPENAI_API_KEY'],
        api_version     = "2023-07-01-preview",
        azure_endpoint  = "https://ai-proxy.lab.epam.com",
        model           = "gpt-4o-mini-2024-07-18",
        temperature     = 0.0
    )

Once you've initialized the LLM of your choice, we can try using it! Let's ask it what LangSmith is - this is something that wasn't present in the training data so it shouldn't have a very good response.

In [3]:
epam_dial.invoke("Tell me a dad joke about programming")

AIMessage(content='Why do programmers prefer dark mode?\n\nBecause light attracts bugs!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 14, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-By611Ms0Mdl5NyRwJdaI8flBTg0mT', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run--fb9ad90a-b7c7-4c00-81a7-abb020d660af-0', usage_metadata={'input_tokens': 14, 'output_tokens': 13, 'total_tokens': 27, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

We can also guide its response with a prompt template. Prompt templates convert raw user input to better input to the LLM.

In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class comedian that tells the longest jokes ever."),
    ("user", "{input}")
])

We can now combine these into a simple LLM chain:

In [5]:
chain = prompt | epam_dial 

We can now invoke it and ask the same request. It should respond in a more longer form!

In [6]:
chain.invoke({"input": "Tell me a joke about software developers"})

AIMessage(content='Alright, settle in because this is going to be a long one!\n\nSo, there’s this software developer named Dave. Now, Dave is your typical coder: he wears glasses, has a collection of T-shirts with obscure programming jokes, and drinks way too much coffee. One day, he decides he’s had enough of working from home and wants to go out and meet people. So, he signs up for a local coding boot camp to learn some new skills and, hopefully, make some friends.\n\nOn the first day of the boot camp, the instructor walks in and says, “Welcome, everyone! Today, we’re going to learn about the importance of teamwork in software development.” Dave, being a bit of a lone wolf, thinks to himself, “Teamwork? I can code just fine on my own, thank you very much!” But he keeps quiet because he’s trying to make a good impression.\n\nThe instructor then announces, “We’re going to split into teams and work on a project together. You’ll have to build a simple app by the end of the week.” Dave’s 

The output of a ChatModel (and therefore, of this chain) is a message. However, it's often much more convenient to work with strings. Let's add a simple output parser to convert the chat message to a string. This is great example of how to use chains.

In [7]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

We can now add this to the previous chain:

In [8]:
chain = prompt | epam_dial | output_parser

We can now invoke it and ask similar question. The answer will now be a string (rather than a ChatMessage).

In [9]:
chain.invoke({"input": "Tell me a joke about software bootcamps"})

'Alright, settle in because this is going to be a long one!\n\nSo, there’s this guy named Dave who decided he wanted to change his life. He was tired of his mundane job as a barista at a coffee shop, where the most exciting thing that happened was when someone ordered a “double shot, half-caf, soy milk, no foam, extra hot, with a sprinkle of cinnamon” – and let me tell you, that was a mouthful! So, one day, while he was pouring a latte, he overheard two customers talking about a software bootcamp. They were raving about how they went from zero to hero in just a few months, making six figures and living the dream. Dave thought, “Why not? I can do that!”\n\nSo, he signed up for this bootcamp, which was advertised as “the fastest way to become a software developer.” The first day, he walked into the classroom, and it was filled with people who looked like they had just come from a tech convention. There were hipsters with beards that could house small animals, and women with colorful hair